## 웹 크롤링

### 인터넷 접속 라이브러리 추가

- urllib.request 모듈로 작업
- html 소스 로드로 별다른 문제 없음

In [84]:
# 1. urllib 호출
from urllib.request import urlopen, Request     #d5 uralib 불러오기 <<< 참고
# Request 사용하면 응답(response)가 와서 보다 안전

# 2. 도시별 날씨 검색함수
def get_weather(city):
    # 기상청 홈페이지 - 도시별 날씨 페이지
    url = 'https://www.weather.go.kr/w/obs-climate/land/city-obs.do'
    page = urlopen(url=url)

    text = page.read().decode('utf-8')
    text = text[text.find(f'>{city}</a>'):]

# 기온 가져오기
    for i in range(7):          # 총 13 raw
        text = text[text.find(f'<td>')+1:]

    start = 3                   # # 3 번째 이후 인덱스 ~ ((( 'td>' 제외하고 읽기)))
    end = text.find('</td>')
    current_temp = text[start:end]

    print(f'{city}의 현재 기온은 {current_temp}˚C 입니다')


# 습도 가져오기
    for i in range(3):          # 7 + '3' => '10' 위치
        text = text[text.find(f'<td>')+1:]

    start = 3                   
    end = text.find('</td>')     
    current_hum = text[start:end]

    print(f'{city}의 현재 기온은 {current_hum}% 입니다')


# 3.
get_weather('부산')

부산의 현재 기온은 11.4˚C 입니다
부산의 현재 기온은 23% 입니다


### OpenAPI 크롤링

- 공공 데이터 포털
    - https://www.data.go.kr
    - 부산광역시 버스정보안내기 현황

    ```python
    response = requests.get(total_url, verify=False)
    ```
    - 문제점
        - https 를 일반 request로 부르면 SSL 오류 발생
        - 해결방법; 외부모듈 requests를 사용, verify=Flase 옵션 지정    # 검증
        - ^ 아래 셀의 19 라인

In [85]:
import requests
from urllib.parse import quote, unquote, urlencode                 # 한글을 URL encode 변환하는 함수 // 롯데 -> 
import json
import ssl

# 2. 

# stationName = 정류소 이름
def getDataSearch(stationName, type):
    api_url = 'https://apis.data.go.kr/6260000/BusanTblBusinfoeqStusService/getTblBusinfoeqStusInfo'
    # 'serviceKey' : Decoding Key
    queryString = "?" + urlencode(
        {
            'serviceKey' : 'nCm3nRGTwdBdzLV9edtOZ7YExTaj/G8IDt2pCwiXRZG8DQP1DLaVHXOJglvIdqym11+OGO4lDr8VFI12IuJxRQ==',
            'pageNo' : '1',
            'numOfRows' : '10',
            'resultType' : type,
            'stationLoc' : stationName
        }
    )
# 3.
    total_url = api_url + queryString
    #print(total_url)
    response = requests.get(total_url, verify=False)
    return response.text

try:
    result = getDataSearch('운동장', 'json')
    # 4. 
    json_data = json.loads(result)

    station_data = json_data['getTblBusinfoeqStusInfo']['body']['items']['item']

    for item in station_data:
        print(item)

except Exception as e:
    print('찾는 데이터가 없습니다')

{'stationNum': '13019', 'stationLoc': '창신초등학교.종합운동장역', 'lat': '35.19244804', 'lng': '129.0673839', 'addr': '연제구 아시아드대로 88', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '06126', 'stationLoc': '사직운동장', 'lat': '35.19516761', 'lng': '129.0651867', 'addr': '동래구 사직1동 101-7', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '06129', 'stationLoc': '사직운동장', 'lat': '35.19640075', 'lng': '129.0651848', 'addr': '동래구 사직1동 97-7', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '02007', 'stationLoc': '구덕운동장', 'lat': '35.11654102', 'lng': '129.0167233', 'addr': '서구 동대신3동 116-1', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '02032', 'stationLoc': '구덕운동장', 'lat': '35.11386333', 'lng': '129.0139233', 'addr': '서구 서대신3동 249-1', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '02044', 'stationLoc': '구덕운동장', 'lat': '35.11331345', 'lng': '129.0162525', 'addr': '서구 서대신동3가 387', 'insYear': '', 'dataDay': '2023-02-03'}


c:\Source\studyPython2023\venv\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


### OpenAPI로 가져온 데이터 지도 표시

```python
    stop_str = '<h4>' + item['stationLoc'] + '</h4>' + item['stationNum'] + '<br>' + item['addr']
```

- 문제점
    - 딕셔너리 구조하고 문자열 포맷팅 f'{}' 호환 안 됨
    - 해결방법; 구식방법인 문자열 결합방식

In [86]:
import folium

if len(station_data) > 0:       # 정류소 중 제일 첫번째 인덱스 정류소 위경도를 중심으로 잡음
    center_lat = station_data[0]['lat']
    center_lng = station_data[0]['lng']

m = folium.Map(location = [center_lat, center_lng], zoom_start=12)

# 전체 정류소 위치값 마커 표시

for item in station_data:

    stop_str = '<h4>' + item['stationLoc'] + '</h4>' + item['stationNum'] + '<br>' + item['addr']

    iframe = folium.IFrame(stop_str)
    popup = folium.Popup(iframe, min_width=200, max_width=200)      # 팝업 사이즈 지정

    folium.Marker(location=[item['lat'], item['lng']], popup=popup,
                  icon = folium.Icon(icon='pushpin')).add_to(m)

m

### Beautiful Soup  (version 4)

웹 크롤링을 편하게 해주는 도구(라이브러리)


```
pip install beautifaulsoup4
```

In [87]:
!pip install beautifulsoup4


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


#### 모듈 import

```python
from bs4 import BeautifulSoup
```

In [88]:
from bs4 import BeautifulSoup
import requests

url = 'https://kin.naver.com/search/list.naver?query=%EC%A0%9C%EC%9E%84%EC%8A%A4%EC%9B%B9+%EC%9A%B0%EC%A3%BC%EB%A7%9D%EC%9B%90%EA%B2%BD'

response = requests.get(url)

if response.status_code == 200:     # webpage OK (200: 정상 / 400: 오류 / 500: 웹서버 오류)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')       # 파싱해라~
#    print(soup)
    title = soup.select_one('#s_content > div.section > ul > li:nth-child(1) > dl > dt > a ')
#    print(title)
    print(title.get_text())

else:
    print(f'Error : {response.status_code}')

#response.text



제임스 웹 망원경 우주사진 원본 어디서... 


In [89]:
from bs4 import BeautifulSoup
import requests

url = 'https://kin.naver.com/search/list.naver?query=%EC%A0%9C%EC%9E%84%EC%8A%A4%EC%9B%B9+%EC%9A%B0%EC%A3%BC%EB%A7%9D%EC%9B%90%EA%B2%BD'

response = requests.get(url)

if response.status_code == 200:     # webpage OK (200: 정상 / 400: 오류 / 500: 웹서버 오류)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')       # 파싱해라~
    ul = soup.select_one('ul.basic1')
    titles = ul.select('li > dl > dt > a')
#    print(titles)
    for title in titles:
        print(title.get_text())

else:
    print(f'Error : {response.status_code}')

#response.text



제임스 웹 망원경 우주사진 원본 어디서... 
제임스 웹 우주망원경이 찍은 첫 사진은... 
제임스 웹 우주망원경
제임스웹 우주망원경 질문이요
제임스웹 망원경
제임스 웹 망원경 촬영 원리
제임스 웹 망원경의 종류 알려주세요
제임스웹 우주망원경
제임스웹우주망원경
제임스웹 우주망원경
